In [1]:
import torch
import torch.nn as nn
import torchtext
torchtext.disable_torchtext_deprecation_warning()

from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F

import random, math, time

import numpy as np 
import pandas as pd
import re

import os
import chardet

import datasets
import gc
import spacy 
import nltk

import stanza

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
torch.__version__

'2.3.0+cu121'

In [4]:
# Download Sindhi model
stanza.download('sd')

2025-02-02 21:50:12 INFO: Downloaded file to C:\Users\Mir Ali\stanza_resources\resources.json
2025-02-02 21:50:12 INFO: Downloading default packages for language: sd (Sindhi) ...
2025-02-02 21:50:13 INFO: File exists: C:\Users\Mir Ali\stanza_resources\sd\default.zip
2025-02-02 21:50:16 INFO: Finished downloading models and saved to C:\Users\Mir Ali\stanza_resources


In [5]:
# Load the Sindhi pipeline with tokenizer
nlp = stanza.Pipeline(lang='sd', processors='tokenize')

2025-02-02 21:50:16 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-02 21:50:16 INFO: Downloaded file to C:\Users\Mir Ali\stanza_resources\resources.json
2025-02-02 21:50:16 INFO: Loading these models for language: sd (Sindhi):
| Processor | Package |
-----------------------
| tokenize  | isra    |

2025-02-02 21:50:16 INFO: Using device: cuda
2025-02-02 21:50:16 INFO: Loading: tokenize
2025-02-02 21:50:18 INFO: Done loading processors!


In [6]:
# Load the English and Sindhi sentences
with open("corpus/NLLB.en-sd.en", "r", encoding="utf-8") as f:
    english_sentences = f.readlines()

with open("corpus/NLLB.en-sd.sd", "r", encoding="utf-8") as f:
    sindhi_sentences = f.readlines()

# Ensure both files have the same number of sentences
assert len(english_sentences) == len(sindhi_sentences), "Mismatch in the number of sentences"

# Create a list of sentence pairs
parallel_corpus = list(zip(english_sentences, sindhi_sentences))

# Example: Print the last 3 sentence pairs
for en, sd in parallel_corpus[-3:]:
    print(f"English: {en.strip()}")
    print(f"Sindhi: {sd.strip()}")
    print()

English: your safety or for the function of the appliance.
Sindhi: هن جي پنهنجي فنڪشن لاءِ ، يا هڪ مناسب فنڪشن لاءِ.

English: ‡Coded as 1 (female) or 2 (male).
Sindhi: طرح 1 (الماس), طرح 2 (طلا)

English: ドキドキ: day after day &
Sindhi: سانچو:عمر سالن ۽ ڏينهن ۾/ڏينهن/دستاويز



In [7]:
# Load the alignment scores
with open("corpus/NLLB.en-sd.scores", "r", encoding="utf-8") as f:
    scores = [float(line.strip()) for line in f]

In [8]:
# Calculate the 90th percentile score
threshold = np.percentile(scores, 97)
print("97th percentile threshold:", threshold)

# Filter the dataset
filtered_corpus = [(en, sd) for en, sd, score in zip(english_sentences, sindhi_sentences, scores) if score > threshold]
print(f"Filtered corpus size: {len(filtered_corpus)}")

97th percentile threshold: 1.100261206
Filtered corpus size: 267741


In [9]:
# Save the filtered corpus to new files
with open("filtered_en.txt", "w", encoding="utf-8") as f_en, open("filtered_sd.txt", "w", encoding="utf-8") as f_sd:
    for en, sd in filtered_corpus:
        f_en.write(en)
        f_sd.write(sd)

In [10]:
def stanza_sindhi_tokenizer(text):
    """Tokenizes Sindhi text using Stanza."""
    doc = nlp(text)  # Process text using Stanza
    tokens = [word.text for sentence in doc.sentences for word in sentence.words]  # Extract tokens
    return tokens

# Example usage
text1 = "زندگي هڪ سفر آهي"
tokens1 = stanza_sindhi_tokenizer(text1)

text2 = "آئون اسڪول وڃان ٿو."
tokens2 = stanza_sindhi_tokenizer(text2)

print(tokens1)
print(tokens2)

['زندگي', 'هڪ', 'سفر', 'آهي']
['آئون', 'اسڪول', 'وڃان', 'ٿو', '.']


In [11]:
# Load language models
nlp_en = spacy.load("en_core_web_sm")  # English

# Tokenize a sentence
english_sentence = "This is a test."
sindhi_sentence = "هي هڪ ٽيسٽ آهي."

# Tokenize English using SpaCy
doc_en = nlp_en(english_sentence)
tokenized_en = [token.text for token in doc_en]

# Tokenize Sindhi using Stanza
tokenized_sd =stanza_sindhi_tokenizer(sindhi_sentence)

print("Tokenized English:", tokenized_en)
print("Tokenized Sindhi:", tokenized_sd)

Tokenized English: ['This', 'is', 'a', 'test', '.']
Tokenized Sindhi: ['هي', 'هڪ', 'ٽيسٽ', 'آهي', '.']


In [13]:
from datasets import Dataset, DatasetDict

# Convert the filtered corpus into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "translation": [
        {"en": en.strip(), "sd": sd.strip()} for en, sd in filtered_corpus
    ]
})
print(dataset)

Dataset({
    features: ['translation'],
    num_rows: 267741
})


In [14]:
# Split the dataset into train, validation, and test sets
train_test_split = dataset.train_test_split(test_size=0.05)  # 5% for the test set

# Further split the test set into validation and test sets (e.g., 50% each)
validation_test_split = train_test_split['test'].train_test_split(test_size=0.5)

# Combine all splits into a DatasetDict
dataset = DatasetDict({
    "train": train_test_split['train'],
    "validation": validation_test_split['train'],
    "test": validation_test_split['test']
})
# Print the resulting DatasetDict
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 254353
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 6694
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 6694
    })
})


In [15]:
# Importing necessary module
from numpy.random import default_rng

# Initializing a random number generator with a specified seed
rng = default_rng(seed=SEED)

# Selecting a random sample of indices from the training dataset
selected_size = 500
select_index = rng.choice(len(dataset['train']), size= selected_size, replace=False)

# Filtering the training dataset based on the selected indices
# This ensures that only the randomly selected subset of data is retained
dataset['train'] = dataset['train'].filter(lambda data, index: index in select_index, with_indices=True)

val_test_size = 100 
select_index_val = rng.choice(len(dataset['validation']), size=val_test_size, replace=False)
select_index_test = rng.choice(len(dataset['test']), size=val_test_size, replace=False)

dataset['validation'] = dataset['validation'].filter(lambda data, index: index in select_index_val, with_indices=True)
dataset['test'] = dataset['test'].filter(lambda data, index: index in select_index_test, with_indices=True)


# Define a lambda function to extract the translation column for a specific language
get_new_col = lambda data, lang: {lang: data['translation'][lang]}

# Map the lambda function to create a new column for Sindhi translation
dataset = dataset.map(get_new_col, fn_kwargs={'lang': "sd"})

# Map the lambda function to create a new column for English translation and remove the original translation column
dataset = dataset.map(get_new_col, remove_columns=['translation'], fn_kwargs={'lang': "en"})

Filter:   0%|          | 0/254353 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6694 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6694 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
train_size = len(list(iter(dataset['train'])))
train_size

500

In [17]:
# Place-holders
token_transform = {}
vocab_transform = {}

SRC_LANGUAGE = 'en'
TRG_LANGUAGE = 'sd'

## Importing of tokenizer libraries 
token_transform["en"] = get_tokenizer('spacy', language='en_core_web_sm')

token_transform["sd"] = stanza_sindhi_tokenizer  

In [20]:
# Example sentence from the dataset
sindhi_example = dataset['train']['sd'][111]
sentence_english = dataset['train']['en'][111]

print("Sentence (English): ", sentence_english)
print("Tokenization (English): ", token_transform['en'](sentence_english)) 

print("Sentence (Sindhi): ", sindhi_example)
print("Tokenization (Sindhi): ", token_transform['sd'](sindhi_example))  

Sentence (English):  "It just takes time to get everything where it needs to be.
Tokenization (English):  ['"', 'It', 'just', 'takes', 'time', 'to', 'get', 'everything', 'where', 'it', 'needs', 'to', 'be', '.']
Sentence (Sindhi):  "ڇاڪاڻ ته هن وقت لتا جو هتي هُجڻ تمام ضروري آهي.
Tokenization (Sindhi):  ['"ڇاڪاڻ', 'ته', 'هن', 'وقت', 'لتا', 'جو', 'هتي', 'هُجڻ', 'تمام', 'ضروري', 'آهي', '.']


In [24]:
def yield_tokens(data, language):
    language_index = {'sd': 0, 'en': 1}  # Update based on your dataset structure

    for data_sample in data:
        yield token_transform[language](data_sample[language])  # Access 'sd' or 'en' based on the `language` paramete
        
# Define special symbols and indices
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']


from torchtext.vocab import build_vocab_from_iterator

# Build vocab for each language ('sd' and 'en')
for ln in ['sd', 'en']:
    vocab_transform[ln] = build_vocab_from_iterator(
        yield_tokens(dataset['train'], ln),
        min_freq=2,  # Set minimum frequency to avoid treating infrequent tokens as UNK
        specials=special_symbols,
        special_first=True
    )

# Set UNK_IDX as the default index
for ln in ['sd', 'en']:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [22]:
print(dataset.keys())


dict_keys(['train', 'validation', 'test'])


In [25]:
print(dataset['train'][0])  # For first sample in training set


{'sd': 'جڏهن جڏهن به مِلو ٿا ڏئي ٿا زخم وڃو،', 'en': 'Whenever you suffer an injury.'}


In [26]:
torch.save(vocab_transform, 'vocab.pth')

# Verify immediately after saving
loaded_vocab_check = torch.load('vocab.pth')

print("🔹 After Saving & Reloading:")
print("English vocab size:", len(loaded_vocab_check['en']))
print("Sindhi vocab size:", len(loaded_vocab_check['sd']))

assert len(loaded_vocab_check['en']) == len(vocab_transform['en']), "Mismatch after saving!"
assert len(loaded_vocab_check['sd']) == len(vocab_transform['sd']), "Mismatch after saving!"
print("✅ Vocabulary is correctly saved and reloaded!")

🔹 After Saving & Reloading:
English vocab size: 528
Sindhi vocab size: 679
✅ Vocabulary is correctly saved and reloaded!


In [27]:
print("Vocabulary keys (languages):", vocab_transform.keys())  # Should be ['en', 'sd'] or similar

Vocabulary keys (languages): dict_keys(['sd', 'en'])


In [28]:
print("English Vocabulary Size:", len(vocab_transform['en']))
print("Sindhi Vocabulary Size:", len(vocab_transform['sd']))

English Vocabulary Size: 528
Sindhi Vocabulary Size: 679


In [29]:
print(vocab_transform.keys())  # Should print dict_keys(['en', 'sd'])

dict_keys(['sd', 'en'])


In [30]:
print("Vocabulary size:", len(vocab_transform))

Vocabulary size: 2


In [31]:
print(vocab_transform[TRG_LANGUAGE]['آئون'])
print(vocab_transform[TRG_LANGUAGE]['پڻ'])

91
370


In [32]:
#we can reverse it....
mapping = vocab_transform[SRC_LANGUAGE].get_itos()

#print 111, for example
mapping[123]

'world'

In [33]:
#let's try unknown vocab
mapping[0]
#they will all map to <unk> which has 0 as integer

'<unk>'

In [34]:
#let's try special symbols
mapping[1], mapping[2], mapping[3]

('<pad>', '<sos>', '<eos>')

In [35]:
#check unique vocabularies
len(mapping)

528

In [36]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

BATCH_SIZE = 64

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([SOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and trg language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TRG_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_batch(batch):
    src_batch, src_len_batch, trg_batch = [], [], []
    for src_sample, trg_sample in batch:
        processed_text = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        src_batch.append(processed_text)
        trg_batch.append(text_transform[TRG_LANGUAGE](trg_sample.rstrip("\n")))
        src_len_batch.append(processed_text.size(0))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first = True) #<----need this because we use linear layers mostly
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first = True)
    return src_batch, torch.tensor(src_len_batch, dtype=torch.int64), trg_batch

In [43]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sd', 'en'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['sd', 'en'],
        num_rows: 100
    })
    test: Dataset({
        features: ['sd', 'en'],
        num_rows: 100
    })
})

In [38]:
# Set the batch size for data loaders
batch_size = 64

train_loader = DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(dataset['validation'],  batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader  = DataLoader(dataset['test'],  batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [ ]:
for en, _, de in train_loader:
    break

In [46]:
print("English shape: ", en.shape)  # (batch_size, seq len)
print("Sindhi shape: ", sd.shape)   # (batch_size, seq len)

English shape:  torch.Size([64, 3])
Sindhi shape:  torch.Size([64, 3])
